# ETL del catálogo de la SSPM

## Generación del catálogo

### Setup

In [8]:
import generate_catalog
import json
import pandas as pd
import os
from openpyxl import load_workbook

In [38]:
#completar con lo que corresponda
catalog_xlsx_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/catalogo-sspm.xlsx"
catalog_json_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"
REPORTES_DIR = os.path.join("/Users/abenassi/github/series-tiempo", "catalogo", "datos", "reportes")

In [35]:
def find_ws_name(wb, name):
    if type(wb) == str or type(wb) == unicode:
        wb = load_workbook(wb, read_only=True, data_only=True)
    
    for sheetname in wb.sheetnames:
        if sheetname.lower() == name.lower():
            return sheetname
    
    return None

In [36]:
field_cols = [
    "field_title", "field_id", "field_identifierCell", "field_dataStartCell",
    "dataset_identifier",     "dataset_title",
    "distribution_identifier", "distribution_title"
]
distribution_cols = [
    "distribution_iedFileURL", "distribution_iedFileSheet", "distribution_identifier",
    "distribution_title", "dataset_identifier",     "dataset_title"
]

In [14]:
df_distribution = pd.read_excel(catalog_xlsx_path, find_ws_name(catalog_xlsx_path, "distribution"))[distribution_cols]
df_field = pd.read_excel(catalog_xlsx_path, find_ws_name(catalog_xlsx_path, "field"))[field_cols]

In [37]:
merged = df_field[field_cols].merge(
    df_distribution[distribution_cols], how="outer", indicator=True, on=[
        "dataset_identifier", "dataset_title", "distribution_identifier", "distribution_title"]
)

In [39]:
merged[merged['_merge'] == 'left_only'].to_excel(
        os.path.join(REPORTES_DIR, "reporte-fields-sin-distribution.xlsx"),
        encoding="utf-8", index=False)
merged[merged['_merge'] == 'right_only'].to_excel(
        os.path.join(REPORTES_DIR, "reporte-distribution-sin-fields.xlsx"),
        encoding="utf-8", index=False)

In [40]:
df_etl_params = merged[merged['_merge'] == 'both']

### Main steps

In [26]:
# leo catalogo en excel
catalog_xlsx = generate_catalog.read_xlsx_catalog(catalog_xlsx_path)

# escribo catalogo en json
generate_catalog.write_json_catalog(catalog, catalog_json_path)

# genero reportes de validación y catálogo filtrado
catalog_filtered = generate_catalog.validate_and_filter(catalog)

# escribo catalogo en json
generate_catalog.write_json_catalog(catalog_filtered, catalog_json_path)


Se encontro un dataset con ID '2', pero su titulo es 'Oferta y Demanda Globales [Base 1993]' en lugar del
esperado '#NAME?'. Este dataset NO se considerara.
No hay ningun dataset con el identifier 2
No se encontro el dataset '#NAME?' especificado para el campo
'demanda_global_ibif_equipo_durable_produccion_material_transporte_nacional' (fila #30 de la hoja "Field"). Este campo no figurara en el data.json de salida.

Se encontro un dataset con ID '4', pero su titulo es 'Oferta y Demanda Globales [Base 2004]' en lugar del
esperado '#NAME?'. Este dataset NO se considerara.
No hay ningun dataset con el identifier 4
No se encontro el dataset '#NAME?' especificado para el campo
'demanda_global_total' (fila #130 de la hoja "Field"). Este campo no figurara en el data.json de salida.


### Pruebas y chequeos

In [37]:
catalog_sspm = generate_catalog.readers.read_catalog(catalog_json_path)[0]

In [42]:
len(catalog_sspm["dataset"]), len(catalog["dataset"]), len(catalog_filtered[0]["dataset"])

(34, 34, 34)

In [27]:
type(catalog), type(catalog_filtered)

(dict, list)

## Scrape datasets IED

### Setup

In [3]:
import os
from xlseries import XlSeries
import pandas as pd
import pydatajson.readers
from copy import deepcopy
from scrape_datasets import scrape_distribution, get_distribution_metadata, get_dataset_metadata, scrape_dataset, scrape_file

In [4]:
catalog_json_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"
etl_params_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/etl_params.csv"
ied_data_dir = "/Users/abenassi/github/series-tiempo/catalogo/datos/ied/"

In [5]:
DATASETS_DIR = "/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/"
REPORTES_DIR = "/Users/abenassi/github/series-tiempo/catalogo/datos/reportes/"

### Main steps

In [6]:
ied_xlsx_path = "/Users/abenassi/github/series-tiempo/catalogo/datos/ied/apendice4.xlsx"

In [7]:
etl_params = pd.read_csv(etl_params_path, dtype={"distribution_identifier": "str"})

In [8]:
catalog = pydatajson.readers.read_catalog(catalog_json_path)

In [9]:
%time report_datasets, report_distributions = scrape_file(ied_xlsx_path, etl_params, catalog, replace=False)

Distribución 1.1: OK (Skipped)
Distribución 1.2: OK (Skipped)
Distribución 2.1: OK (Skipped)
Distribución 2.2: OK (Skipped)
Distribución 2.3: OK (Skipped)
Distribución 2.4: OK (Skipped)
Distribución 3.1: OK (Skipped)
Distribución 3.2: ERROR (AssertionError('End must be greater than start!',))
Distribución 4.1: OK (Skipped)
Distribución 4.2: OK (Skipped)
Distribución 4.3: OK (Skipped)
Distribución 4.4: OK (Skipped)
Distribución 5.1: ERROR (FieldTitleRepetitionError(u"Hay mas de 1 field con title indice_tiempo: [{u'dataStartCell': u'A7', u'title': u'indice_tiempo', u'specialTypeDetail': u'R/P1Y', u'specialType': u'time_index', u'identifierCell': u'A7', u'type': u'date', u'id': u'5.1_IT_1993_ A_13'}, {u'dataStartCell': u'A8', u'title': u'indice_tiempo', u'specialTypeDetail': u'R/P3M', u'specialType': u'time_index', u'identifierCell': u'A7', u'type': u'date', u'id': u'5.1_IT_1993_ T_13'}]",))
Distribución 5.2: ERROR (IndexError('tuple index out of range',))
Distribución 6.1: OK (Skipped)
D

In [10]:
report_distributions.groupby("distribution_status")[["distribution_identifier"]].count()

,distribution_identifier
distribution_status,
ERROR,33
OK,52


In [11]:
cols_rep_dataset = ["distribution_iedFileURL", "dataset_identifier", "dataset_status"]
report_datasets[cols_rep_dataset].to_csv(os.path.join(REPORTES_DIR, "reporte-datasets-scraping.csv"), 
                       encoding="utf-8", index=False)

cols_rep_distribution = ["distribution_iedFileURL", "dataset_identifier", 
                         "distribution_identifier", "distribution_status", "distribution_notes"]
report_distributions[cols_rep_distribution].to_csv(os.path.join(REPORTES_DIR, "reporte-distributions-scraping.csv"), 
                            encoding="utf-8", index=False)

### Pruebas y chequeos

In [9]:
# cargo el excel de ied
xl = XlSeries(ied_xlsx_path)
ied_xlsx_filename = os.path.basename(ied_xlsx_path)

In [10]:
distribution = scrape_distribution(xl, etl_params, catalog, "106.2")

AssertionError: No end could be estimated! End: None   | Start: 36

In [ ]:
df_distrib = etl_params[
        etl_params.distribution_identifier == "3.2"
    ]

In [ ]:
df_distrib

In [42]:
result = scrape_dataset(xl, etl_params, catalog, "2", debug_mode=True)

In [48]:
result

"AssertionError('End must be greater than start!',)"

In [19]:
distribution.to_csv("/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/1/63.csv", 
                    encoding="utf-8-sig", index_label="indice_tiempo")

In [17]:
distrib_meta = deepcopy(get_distribution_metadata(catalog, "106.2"))

In [18]:
distrib_meta

{u'accessURL': u'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
 u'description': u'\xcdndice de Precios Internos al por Mayor. Base 1993. Valores trimestrales.',
 u'field': [{u'dataStartCell': u'A35',
   u'id': u'106.2_IT_1993_T_13',
   u'identifierCell': u'A34',
   u'specialType': u'time_index',
   u'specialTypeDetail': u'R/P3M',
   u'title': u'indice_tiempo',
   u'type': u'date'},
  {u'dataStartCell': u'B35',
   u'description': u'IPIM Nivel general. Base 1993. Trimestral',
   u'id': u'106.2_I1NG_1993_T_23',
   u'identifierCell': u'B34',
   u'specialTypeDetail': u'',
   u'title': u'ipim_1993_nivel_general',
   u'type': u'number',
   u'units': u'N\xfamero \xedndice'},
  {u'dataStartCell': u'C35',
   u'description': u'IPIM Total productos nacionales. Base 1993. Trimestral',
   u'id': u'106.2_I1TPN_1993_T_36',
   u'identifierCell': u'C34',
   u'specialTypeDetail': u'',
   u'title': u'ipim_1993_total_productos_nacionales',
   u'type': u'number'

In [22]:
pd.DataFrame(get_distribution_metadata(catalog, "106.2")).iloc[0,:]["iedFileURL"]

u'http://www.economia.gob.ar/download/infoeco/apendice4.xlsx'

In [32]:
etl_params[etl_params.distribution_identifier == "106.2"]

,distribution_iedFileURL,distribution_iedFileSheet,distribution_timeIndexCell,distribution_identifier,distribution_title,field_title,field_id,field_identifierCell,field_dataStartCell
4695,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,indice_tiempo,106.2_IT_1993_T_13,A34,A35
4696,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_nivel_general,106.2_I1NG_1993_T_23,B34,B35
4697,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_total_productos_nacionales,106.2_I1TPN_1993_T_36,C34,C35
4698,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_productos_nacionales_primarios,106.2_I1PNP_1993_T_40,D34,D35
4699,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_productos_nacionales_manufacturados_...,106.2_I1PNMEE_1993_T_63,E34,E35
4700,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_productos_importados,106.2_IPI_1993_T_25,F34,F35
4701,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_precios_relativos_nacionales_importados,106.2_I1PRNI_1993_T_49,G34,G35
4702,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A35,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_precios_relativos_primarios_manufact...,106.2_I1PRPMEE_1993_T_70,H34,H35
4703,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A36,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_nivel_general_variacion_mensual,106.2_I1NGVM_1993_T_41,J34,J36
4704,http://www.economia.gob.ar/download/infoeco/ap...,4.7 IPIM,A39,106.2,Índice de Precios Internos al por Mayor. Base ...,ipim_1993_nivel_general_variacion_anual,106.2_I1NGVA_1993_T_39,K34,K39


In [27]:
params = gen_distribution_params(etl_params, catalog, "106.2")
dist_meta = get_distribution_metadata(catalog, "106.2")

NameError: name 'gen_distribution_params' is not defined

In [23]:
params

{u'data_starts': [9, 9, 9, 9, 9, 9],
 u'frequency': u'Y',
 u'headers_coord': ['B8', 'C8', 'D8', 'E8', 'F8', 'G8'],
 u'series_names': ['oferta_global_pbi',
  'oferta_global_importacion',
  'demanda_global_exportacion',
  'demanda_global_ibif',
  'demanda_global_consumo_priv',
  'demanda_global_consumo_publico'],
 u'time_header_coord': 'A8',
 u'worksheet': '1.2 OyD real s.e.'}

In [24]:
dist_meta

{u'accessURL': u'http://www.minhacienda.gob.ar/secretarias/politica-economica/programacion-macroeconomica/',
 u'description': u'Oferta y Demanda Globales por componente, a precios de comprador, en millones de pesos de 1993 y valores anuales desestacionalizados.',
 u'field': [{u'dataStartCell': u'A9',
   u'id': u'1.1_IT_ D_1993_ A_13',
   u'identifierCell': u'A8',
   u'specialType': u'time_index',
   u'specialTypeDetail': u'R/P1Y',
   u'title': u'indice_tiempo',
   u'type': u'date'},
  {u'dataStartCell': u'B9',
   u'description': u'PIB desestacionalizado, en millones de pesos de 1993 y valores anuales',
   u'id': u'1.1_OGP_ D_1993_ A_17',
   u'identifierCell': u'B8',
   u'title': u'oferta_global_pbi',
   u'type': u'number',
   u'units': u'Millones de pesos, a precios de 1993'},
  {u'dataStartCell': u'C9',
   u'description': u'Importaciones desestacionalizadas, en millones de pesos de 1993 y valores anuales',
   u'id': u'1.1_OGI_ D_1993_ A_25',
   u'identifierCell': u'C8',
   u'title': u

In [2]:
from pydatajson import readers

In [11]:
catalog = readers.read_catalog(catalog_json_path)

In [15]:
for dataset in catalog["dataset"]:
    for distribution in dataset["distribution"]:
        if "field" in distribution:
            for field in distribution["field"]:
                if field["id"] == "1.1_OGP_D_1993_A_17":
                    print dataset["identifier"], distribution["identifier"], field["id"]

1 1.1 1.1_OGP_D_1993_A_17


In [6]:
elecciones = [dataset for dataset in catalog["dataset"] if dataset["title"] == "Resultados Electorales 2015"][0]

In [9]:
for i in [distribution["downloadURL"] for distribution in elecciones["distribution"]]:
    print i

http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PRES_Y_VICE_.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PARLAMENTARIO_NACIONAL.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PARLAMENTARIO_REGIONAL.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/DIP_NAC.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/FMESSN_0313.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/DIP_PROV.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/SEN_PROV.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/GOBERNADOR.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/MUNICIPAL.rar
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/DEPARTAMENTOS.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/FPARTIDOS.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PROVINCIAS.csv
http://www.elecciones.gob.ar/admin/ckfinder/userfiles/files/PV_0101.csv
http://

In [68]:
def get_time_series(
    dataset_id, distribution_id, field_title, time_index="indice_tiempo",
    datasets_dir="/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/"):
    """Devuelve un dataframe con una serie de tiempo."""
    
    distribution_path = os.path.join(datasets_dir, dataset_id, distribution_id + ".csv")
    
    return pd.read_csv(distribution_path, index_col=time_index, parse_dates=True)[field_title]

In [121]:
def get_time_series_df(
    field_ids, catalog="/Users/abenassi/github/series-tiempo/catalogo/datos/data.json"):
    catalog = readers.read_catalog(catalog)
    
    if not isinstance(field_ids, list):
        field_ids = [field_ids]
    
    # busca los ids de dataset y distribucion de la serie
    series_params = []
    for dataset in catalog["dataset"]:
        for distribution in dataset["distribution"]:
            if "field" in distribution:
                for field in distribution["field"]:
                    if field["id"] in field_ids:
                        
                        series_params.append((
                            dataset["identifier"],
                            distribution["identifier"],
                            field["title"]
                        ))
    
    # toma las series y genera un data frame concatenado
    time_series = [get_time_series(*serie_params) 
                   for serie_params in series_params]

    return pd.concat(time_series, axis=1)

In [132]:
df = get_time_series_df([
    "4.2_OGP_2004_T_17", "23.2_PIBPM_2004_T_38"
])

291.92582963699999

In [138]:
df["oferta_global_pbi_precios_2011_II"] = df.oferta_global_pbi / (df.indice_precios_implicito_pbi_mercado / df.indice_precios_implicito_pbi_mercado["2011-04-01"])

In [128]:
df["demanda_global_ibif_total / ofert_global_pbi"] = df["demanda_global_ibif_total"] / df["oferta_global_pbi"]

In [141]:
df.to_csv("/Users/abenassi/github/series-tiempo/ejemplos/deflacion_serie_por_otra.csv", encoding="utf8")

In [142]:
ipc = get_time_series_df("104.1_I2NG_2016_M_22")

## Upload data y metadata

## Setup

In [172]:
import pysftp
import requests
import yaml

In [173]:
cnopts = pysftp.CnOpts()
cnopts.hostkeys.load('/Users/abenassi/.ssh/known_hosts')
cnopts.hostkeys = None 

## Main steps

In [182]:
def upload_file_to_ind(local_path, remote_dir=None, config_ind="ind/config_ind.yaml"):
    """Carga de un archivo a la infraestructura."""

    if isinstance(config_ind, dict):
        ind_params = config_ind
    else:
        with open(config_ind_path, 'r') as f:
            ind_params = yaml.load(f)
        
    remote_dir = remote_dir or '/home/{}'.format(ind_params["user"])

    with pysftp.Connection(ind_params["host"], username=ind_params["user"], password=ind_params["pass"], cnopts=cnopts) as sftp:
        with sftp.cd(remote_dir):
            sftp.put(local_path)


In [183]:
def upload_datajson_to_ind(local_path, config_ind_path="ind/config_ind.yaml"):
    
    with open(config_ind_path, 'r') as f:
        ind_params = yaml.load(f)
        
    upload_file_to_ind(local_path, config_ind=ind_params)

    headers = {"Authorization": ind_params["api"]["metadata"]["auth_header"]}

    return requests.get(ind_params["api"]["metadata"]["url"], headers=headers)

In [174]:
upload_file_to_ind('/Users/abenassi/github/series-tiempo/catalogo/datos/data.json')

In [185]:
upload_datajson_to_ind('/Users/abenassi/github/series-tiempo/catalogo/datos/data.json')

<Response [200]>

In [39]:
with pysftp.Connection(host, username=user, password=password) as sftp:
    with sftp.cd('/home/abenassi'):             # temporarily chdir to public
        sftp.put('/Users/abenassi/github/series-tiempo/catalogo/datos/datasets/1/63.csv')  # upload file to public/ on remote
        # sftp.get('remote_file')         # get a remote file

In [41]:
url = url_api_ind
headers = {"Authorization": auth_header}
res = requests.get(url, headers=headers)

In [42]:
res.content

'<webresult>\n  <result>OK</result>\n  <message>Job started</message>\n  <id>1ce9bc81-e519-4f7b-b921-ac2c1473e84f</id>\n</webresult>\n\n'

### Pruebas y chequeos

In [109]:
from pydatajson.readers import read_catalog
from pydatajson import DataJson

In [35]:
dj = DataJson("catalog.json", "/Users/abenassi/github/series-tiempo/catalogo/codigo/schemas/")

In [36]:
dj.is_valid_catalog("/Users/abenassi/github/series-tiempo/catalogo/datos/data.json")

True

In [110]:
catalog = read_catalog("/Users/abenassi/github/series-tiempo/catalogo/datos/data.json")

In [7]:
catalog[0]["title"]

u'Datos Programaci\xf3n Macroecon\xf3mica'

### Chequeos webdav

In [5]:
import requests
from requests.auth import HTTPBasicAuth

In [6]:
res = requests.get("https://meconcd.mecon.gob.ar/remote.php/webdav/", 
                   auth=HTTPBasicAuth(options["webdav_login"], options["webdav_password"]), verify=False)

/Users/abenassi/anaconda/envs/series-tiempo/lib/python2.7/site-packages/urllib3/connectionpool.py:852: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [9]:
import pycurl

In [32]:
res.

<RequestsCookieJar[Cookie(version=0, name='oc41be7089f3', value='sj0jiee5lnk1pov11e98rs6vg7', port=None, port_specified=False, domain='meconcd.mecon.gob.ar', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=None, discard=True, comment=None, comment_url=None, rest={'HttpOnly': None}, rfc2109=False)]>

In [12]:
import easywebdav

In [21]:
webdav = easywebdav.connect('meconcd.mecon.gob.ar/remote.php/webdav/', username='abenassi', password='alpha1g13', protocol='http')

In [22]:
webdav.mkdir('some_dir')

OperationFailed: Failed to create directory "some_dir".
  Operation     :  MKCOL some_dir
  Expected code :  201 Created
  Actual code   :  302 Found

In [10]:
import webdav.client as wc
options = {
 'webdav_hostname': "https://meconcd.mecon.gob.ar/remote.php/webdav/",
 'webdav_login':    "abenassi",
 'webdav_password': "alpha1g13",
  "cert_path": "/Users/abenassi/github/series-tiempo/catalogo/codigo/certificate/AutoridadCertificantedelMinisteriodeEconomiaversionII.crt",
    
 
}
client = wc.Client(options)
client.list()


NotConnection: Not connection with https://meconcd.mecon.gob.ar/remote.php/webdav/

In [14]:
client.mkdir("new_dir")

RemoteParentNotFound: Remote parent for: /new_dir/ not found

In [12]:
client.clean("/")

In [9]:
client.upload_sync(remote_path="helpers.py", local_path="helpers.py")

RemoteParentNotFound: Remote parent for: /helpers.py not found